# SLC25A46 - Genotyping-imputed GP2 data

## Description

GP2 genotyping imputed data: Release 7

### Getting Started

- Loading Python libraries
- Defining functions
- Set paths
- Installing packages

### Making a working directory

### Copy Over Files

### Create a covariate file with GP2 data

### Annotation 

- Turn binary files into VCF
- Annotate using ANNOVAR

### Case/Control Analysis

- All variants (Association and Logistic regression)
- Coding (Association and Logistic regression)

## Getting Started

### Loading Python libraries

In [2]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

### Defining functions

In [3]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("chrom://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name @ `WORKSPACE_NAME`: {WORKSPACE_NAME}')
print(f'Billing Project @ `BILLING_PROJECT_ID`: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data @ `WORKSPACE_BUCKET`: {WORKSPACE_BUCKET}')
print('')

## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'path/'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'
AMP_RELEASE_GATK_PATH = os.path.join(AMP_RELEASE_PATH, 'gatk')
AMP_WGS_RELEASE_PATH = 'path/'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

## GP2 v7.0
## Explicitly define release v7.0 path 
GP2_RELEASE_PATH = 'path/'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'

print('GP2 v7.0')
print(f'Path to GP2 v7.0 Clinical Data @ `GP2_CLINICAL_RELEASE_PATH`: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Metadata @ `GP2_META_RELEASE_PATH`: {GP2_META_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Raw Genotype Data @ `GP2_RAW_GENO_PATH`: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v7.0 Imputed Genotype Data @ `GP2_IMPUTED_GENO_PATH`: {GP2_IMPUTED_GENO_PATH}')
print(f'Path to GP2 v7.0 summary statistics: {GP2_SUMSTAT_RELEASE_PATH}')

### Install packages

#### Install Plink 1.9 and Plink 2.0

In [5]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_20240504.zip
unzip -o plink2_linux_x86_64_20240504.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [6]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_x86_64_20240504.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [7]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [8]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [9]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [10]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_x86_64_20240504.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [11]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [12]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## Making a working directory

## Can use any ancestry - using EAS just as example

In [13]:
ancestry = 'EAS'
WORK_DIR = f'SLC25A46_{ancestry}'

! mkdir {WORK_DIR}

## Copy Over Files

In [ ]:
# Check directory where GP2 Tier 2 data is
print("List available imputed genotype information in GP2 (broken down by ancestry)")
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

In [ ]:
##  Imputed genotype GP2 Tier 2 data
# SLC25A46 coordinates: Chromosome chr5:110,738,136-110,765,161 https://www.genecards.org/cgi-bin/carddisp.pl?gene=SLC25A46&keywords=SLC25A46
ancestry = 'EAS'  # CHANGE THE CHROMOSOME DEPENDING ON THE GENE AND THE ANCESTRY
WORK_DIR = f'SLC25A46_{ancestry}'

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_IMPUTED_GENO_PATH}/{ancestry}/chr5_{ancestry}_release7.* {WORK_DIR}')

In [ ]:
## clinical data 

## master key
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_CLINICAL_RELEASE_PATH}/master_key_release7_final.csv {WORK_DIR}')

## related file 
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_META_RELEASE_PATH}/related_samples/{ancestry}_release7.related {WORK_DIR}')

In [ ]:
## PCs
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_RAW_GENO_PATH}/{ancestry}/{ancestry}_release7.eigenvec {WORK_DIR}')

In [18]:
## Check files
! ls {WORK_DIR}

chr5_EAS_release7.log	chr5_EAS_release7.pvar	master_key_release7_final.csv
chr5_EAS_release7.pgen	EAS_release7.eigenvec
chr5_EAS_release7.psam	EAS_release7.related


## Create a covariate file with GP2 data

In [ ]:
# Let's load the master key
key = pd.read_csv(f'{WORK_DIR}/master_key_release7_final.csv')
print(key.shape)
key.head()

In [ ]:
# Subsetting to keep only a few columns 
key = key[['GP2sampleID', 'baseline_GP2_phenotype_for_qc', 'biological_sex_for_qc', 'age_at_sample_collection', 'age_of_onset', 'label']]
# Renaming the columns
key.rename(columns = {'GP2sampleID':'IID',
                                     'baseline_GP2_phenotype_for_qc':'phenotype',
                                     'biological_sex_for_qc':'SEX', 
                                     'age_at_sample_collection':'AGE', 
                                     'age_of_onset':'AAO'}, inplace = True)
key

In [ ]:
## Subset to keep ancestry of interest 
ancestry_key = key[key['label']==ancestry].copy()
ancestry_key.reset_index(drop=True)

In [ ]:
# Load information about related individuals in the ancestry analyzed
related_df = pd.read_csv(f'{WORK_DIR}/{ancestry}_release7.related')
print(related_df.shape)
related_df

In [ ]:
# Make a list of just one set of related people
related_list = list(related_df['IID1'])

# Check value counts of related and remove only one related individual
ancestry_key = ancestry_key[~ancestry_key["IID"].isin(related_list)]

# Check size
print(ancestry_key.shape)
ancestry_key

In [ ]:
# Convert phenotype to binary (1/2)
## Assign conditions so case=2 and controls=1, and -9 otherwise (matching PLINK convention)
    # PD = 2; control = 1
pheno_mapping = {"PD": 2, "Control": 1}
ancestry_key['PHENO'] = ancestry_key['phenotype'].map(pheno_mapping).astype('Int64')
ancestry_key

In [25]:
# Check value counts of pheno
ancestry_key['PHENO'].value_counts(dropna=False)

PHENO
2       2657
1       2572
<NA>      44
Name: count, dtype: Int64

In [ ]:
## Get the PCs
pcs = pd.read_csv(f'{WORK_DIR}/{ancestry}_release7.eigenvec', sep='\t')
pcs.head()

In [ ]:
# Split the single column into multiple columns
#pcs_split = pcs['#FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10'].str.split(',', expand=True)

selected_columns = ['IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
pcs = pd.DataFrame(data=pcs.iloc[:, 1:7].values, columns=selected_columns)

# Drop the first row (since it's now the column names)
pcs = pcs.drop(0)

# Reset the index to remove any potential issues
pcs = pcs.reset_index(drop=True)

# Display the resulting DataFrame
print(pcs)

In [ ]:
# Check value counts of related and remove only one related individual
# pcs = pcs[~pcs["IID"].isin(related_list)]

# Check size
print(pcs.shape)
pcs

In [29]:
# Check value counts of SEX     
ancestry_key['SEX'].value_counts(dropna=False)

SEX
Male                          3388
Female                        1879
Other/Unknown/Not Reported       6
Name: count, dtype: int64

In [ ]:
# Convert phenotype to binary (1/2)
## Assign conditions so female=2 and men=1, and -9 otherwise (matching PLINK convention)
    # Female = 2; Male = 1
sex_mapping = {"Female": 2, "Male": 1}
ancestry_key['SEX'] = ancestry_key['SEX'].map(sex_mapping).astype('Int64')
ancestry_key

In [31]:
# Check value counts of SEX     
ancestry_key['SEX'].value_counts(dropna=False)

SEX
1       3388
2       1879
<NA>       6
Name: count, dtype: Int64

In [32]:
## Make covariate file
df = pd.merge(pcs, ancestry_key, on='IID', how='left')
df.columns

Index(['IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'phenotype', 'SEX', 'AGE',
       'AAO', 'label', 'PHENO'],
      dtype='object')

In [33]:
df['FID'] = 0

In [ ]:
## Clean up and keep columns we need 
final_df = df[['FID','IID', 'SEX', 'AGE', 'PHENO','PC1', 'PC2', 'PC3', 'PC4', 'PC5']].copy()
final_df

In [35]:
final_df['PHENO'] = final_df['PHENO'].fillna(-9)
final_df['AGE'] = final_df['AGE'].fillna(-9)
final_df['SEX'] = final_df['SEX'].fillna(-9)

In [36]:
final_df['SEX'].value_counts()

SEX
1     3127
2     1805
-9     206
Name: count, dtype: Int64

In [37]:
#Update sex to plink format: 1=male, 2=female

In [ ]:
final_df[(final_df['PHENO']==2)&(final_df['AGE']==-9)]

In [ ]:
final_df[(final_df['PHENO']==1)&(final_df['AGE']==-9)]

In [40]:
## Make file of sample IDs to keep 
samples_toKeep = final_df[['FID', 'IID']].copy()
samples_toKeep.to_csv(f'{WORK_DIR}/{ancestry}.samplestoKeep', sep = '\t', index=False, header=None)

In [41]:
## Make your covariate file
final_df.to_csv(f'{WORK_DIR}/{ancestry}_covariate_file.txt', sep = '\t', index=False)

In [42]:
## check to make sure file was created and saved
! ls {WORK_DIR}

chr5_EAS_release7.log	chr5_EAS_release7.pvar	EAS_release7.related
chr5_EAS_release7.pgen	EAS_covariate_file.txt	EAS.samplestoKeep
chr5_EAS_release7.psam	EAS_release7.eigenvec	master_key_release7_final.csv


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_covariate_file.txt {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/{ancestry}_covariate_file.txt')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}.samplestoKeep {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/{ancestry}.samplestoKeep')


In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/')

## Annotation

In [45]:
### Extract the region using PLINK

In [46]:
## extract region using plink
WORK_DIR = f'SLC25A46_{ancestry}'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr5_{ancestry}_release7 \
--chr 5 \
--from-bp 110738136 \
--to-bp 110765161 \
--make-bed \
--out {WORK_DIR}/{ancestry}_SLC25A46

PLINK v2.00a6LM 64-bit Intel (4 May 2024)      www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.log.
Options in effect:
  --chr 5
  --from-bp 110738136
  --make-bed
  --out SLC25A46_EAS/EAS_SLC25A46
  --pfile SLC25A46_EAS/chr5_EAS_release7
  --to-bp 110765161

Start time: Thu May 16 02:57:54 2024
3676 MiB RAM detected, ~1456 available; reserving 1392 MiB for main workspace.
Using 1 compute thread.
5139 samples (1870 females, 3269 males; 5139 founders) loaded from
SLC25A46_EAS/chr5_EAS_release7.psam.
11444438 variants loaded from SLC25A46_EAS/chr5_EAS_release7.pvar.
1 binary phenotype loaded (2646 cases, 2453 controls).
1951 variants remaining after main filters.
Writing SLC25A46_EAS/EAS_SLC25A46.fam ... done.
Writing SLC25A46_EAS/EAS_SLC25A46.bim ... done.
Writing SLC25A46_EAS/EAS_SLC25A46.bed ... done.
End time: Thu May 16 02:58:05 2024


In [47]:
# Visualize bim file
! head {WORK_DIR}/{ancestry}_SLC25A46.bim

5	chr5:110738152:C:T	0	110738152	T	C
5	chr5:110738155:C:G	0	110738155	G	C
5	chr5:110738163:A:G	0	110738163	G	A
5	chr5:110738172:G:A	0	110738172	A	G
5	chr5:110738251:G:A	0	110738251	A	G
5	chr5:110738257:G:C	0	110738257	C	G
5	chr5:110738260:G:A	0	110738260	A	G
5	chr5:110738267:A:T	0	110738267	T	A
5	chr5:110738270:A:G	0	110738270	G	A
5	chr5:110738286:CA:C	0	110738286	C	CA


In [ ]:
# Visualize fam file
! head {WORK_DIR}/{ancestry}_SLC25A46.fam

In [49]:
# Visualize bed file
#! head {WORK_DIR}/{ancestry}_SLC25A46.bed

### Turn binary files into VCF

In [50]:
## Turn binary files into VCF
! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--recode vcf id-paste=iid \
--mac 2 \
--out {WORK_DIR}/{ancestry}_SLC25A46

PLINK v2.00a6LM 64-bit Intel (4 May 2024)      www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.log.
Options in effect:
  --bfile SLC25A46_EAS/EAS_SLC25A46
  --export vcf id-paste=iid
  --mac 2
  --out SLC25A46_EAS/EAS_SLC25A46

Start time: Thu May 16 02:58:12 2024
3676 MiB RAM detected, ~1540 available; reserving 1476 MiB for main workspace.
Using 1 compute thread.
5139 samples (1870 females, 3269 males; 5139 founders) loaded from
SLC25A46_EAS/EAS_SLC25A46.fam.
1951 variants loaded from SLC25A46_EAS/EAS_SLC25A46.bim.
1 binary phenotype loaded (2646 cases, 2453 controls).
Calculating allele frequencies... done.
1742 variants removed due to allele frequency threshold(s)
(--maf/--max-maf/--mac/--max-mac).
209 variants remaining after main filters.
--export vcf to SLC25A46_EAS/EAS_SLC25A46.vcf ... 10101111121213131414151516161717181819192020212122222323242425252626272728282929303031313232333

In [51]:
### Bgzip and Tabix (zip and index the file)

! bgzip -f {WORK_DIR}/{ancestry}_SLC25A46.vcf
! tabix -f -p vcf {WORK_DIR}/{ancestry}_SLC25A46.vcf.gz 

### Annotate using ANNOVAR

In [52]:
## annotate using ANNOVAR
! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/{ancestry}_SLC25A46.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out {WORK_DIR}/{ancestry}_SLC25A46.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 SLC25A46_EAS/EAS_SLC25A46.vcf.gz > SLC25A46_EAS/EAS_SLC25A46.annovar.avinput>
NOTICE: Finished reading 216 lines from VCF file
NOTICE: A total of 209 locus in VCF file passed QC threshold, representing 193 SNPs (123 transitions and 70 transversions) and 16 indels/substitutions
NOTICE: Finished writing allele frequencies based on 991827 SNP genotypes (632097 transitions and 359730 transversions) and 82224 indels/substitutions for 5139 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl SLC25A46_EAS/EAS_SLC25A46.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile SLC25A46_EAS/EAS_SLC25A46.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with 

In [ ]:
# Read in ANNOVAR multianno file
gene = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.annovar.hg38_multianno.txt', sep = '\t')
display(gene)

In [ ]:
gene = gene[gene['Gene.refGene'] == 'SLC25A46']
gene

In [ ]:
# Filter exonic variants
coding = gene[(gene['Func.refGene'] == 'exonic')]
coding

In [56]:
# Filter exonic and non-synonymous variants
coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo5142,Otherinfo5143,Otherinfo5144,Otherinfo5145,Otherinfo5146,Otherinfo5147,Otherinfo5148,Otherinfo5149,Otherinfo5150,Otherinfo5151
13,5,110739207,110739207,G,C,exonic,SLC25A46,.,nonsynonymous SNV,"SLC25A46:NM_001303249:exon1:c.G88C:p.A30P,SLC2...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
15,5,110739387,110739387,C,A,exonic,SLC25A46,.,nonsynonymous SNV,"SLC25A46:NM_001303249:exon1:c.C268A:p.Q90K,SLC...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
183,5,110761398,110761398,G,C,exonic,SLC25A46,.,nonsynonymous SNV,"SLC25A46:NM_001303250:exon8:c.G600C:p.K200N,SL...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [57]:
results = [] 

utr5 = gene[gene['Func.refGene']== 'UTR5']
intronic = gene[gene['Func.refGene']== 'intronic']
exonic = gene[gene['Func.refGene']== 'exonic']
utr3 = gene[gene['Func.refGene']== 'UTR3']
coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] != 'nonsynonymous SNV')]
print({ancestry})
print('Total variants: ', len(gene))
print("Intronic: ", len(intronic))
print('UTR3: ', len(utr3))
print('UTR5: ', len(utr5))
print("Total exonic: ", len(exonic))
print('  Synonymous: ', len(coding_synonymous))
print("  Nonsynonymous: ", len(coding_nonsynonymous))
results.append((gene, intronic, utr3, utr5, exonic, coding_synonymous, coding_nonsynonymous))
print('\n')
    
#output = pd.DataFrame(results, columns=('Total variants','Intronic', 'UTR3','UTR5', 'Total exonic', "Synonymous", 'Nonsynonymous'))

{'EAS'}
Total variants:  209
Intronic:  172
UTR3:  25
UTR5:  3
Total exonic:  9
  Synonymous:  6
  Nonsynonymous:  3




In [58]:
# Save in PLINK format 
variants_toKeep = coding_nonsynonymous[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep.to_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.all_coding_nonsyn.variantstoKeep.txt', sep="\t", index=False, header=False)
variants_toKeep

,Chr,Start,End,Gene.refGene
13,5,110739207,110739207,SLC25A46
15,5,110739387,110739387,SLC25A46
183,5,110761398,110761398,SLC25A46


In [59]:
variants_toKeep.shape

(3, 4)

In [60]:
# Save in PLINK format 
variants_toKeep2 = coding[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep2.to_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.all_coding.variantstoKeep', sep="\t", index=False, header=False)
variants_toKeep2

,Chr,Start,End,Gene.refGene
13,5,110739207,110739207,SLC25A46
14,5,110739266,110739266,SLC25A46
15,5,110739387,110739387,SLC25A46
39,5,110743749,110743749,SLC25A46
40,5,110743781,110743781,SLC25A46
72,5,110748258,110748258,SLC25A46
142,5,110756711,110756711,SLC25A46
182,5,110761239,110761239,SLC25A46
183,5,110761398,110761398,SLC25A46


In [61]:
variants_toKeep2.shape

(9, 4)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_SLC25A46.all_coding_nonsyn.variantstoKeep.txt {WORKSPACE_BUCKET}/UHRF1BP1L_{ancestry}/{ancestry}_UHRF1BP1L.all_coding_nonsyn.variantstoKeep ')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_SLC25A46.all_coding.variantstoKeep {WORKSPACE_BUCKET}/UHRF1BP1L_{ancestry}/{ancestry}_UHRF1BP1L.all_coding.variantstoKeep ')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/')

## Case/Control Analysis

### Glossary
- CHR Chromosome code
- SNP Variant identifier
- A1 Allele 1 (usually minor)
- A2 Allele 2 (usually major)
- MAF Allele 1 frequency in all subjects
- F_A/MAF_A Allele 1 frequency in cases
- F_U/MAF_U Allele 1 frequency in controls
- NCHROBS_A Number of case allele observations
- NCHROBS_U Number of control allele observations

### ALL VARIANTS

#### Association

In [64]:
WORK_DIR = f'SLC25A46_{ancestry}'

! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--assoc \
--allow-no-sex \
--ci 0.95 \
--out {WORK_DIR}/{ancestry}_SLC25A46.all

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.all.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile SLC25A46_EAS/EAS_SLC25A46
  --ci 0.95
  --keep SLC25A46_EAS/EAS.samplestoKeep
  --out SLC25A46_EAS/EAS_SLC25A46.all

3676 MB RAM detected; reserving 1838 MB for main workspace.
1951 variants loaded from .bim file.
5139 people (3269 males, 1870 females) loaded from .fam.
5099 phenotype values loaded from .fam.
--keep: 5138 people remaining.
Using 1 thread.
Before main variant filters, 5138 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.997895.
1951 variants and 5138 people pass f

In [65]:
freq = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.all.assoc', delim_whitespace=True)
sig_all_nonadj = freq[freq['P']<0.05]
sig_all_nonadj

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
85,5,chr5:110738976:T:G,110738976,G,0.027300,0.021250,T,3.909,0.048030,1.2930,0.13020,1.00200,1.6690
108,5,chr5:110739078:G:C,110739078,C,0.000000,0.001428,G,7.554,0.005989,0.0000,inf,0.00000,NaN
184,5,chr5:110739776:AG:A,110739776,A,0.027270,0.021240,AG,3.886,0.048690,1.2920,0.13020,1.00100,1.6680
252,5,chr5:110740651:A:G,110740651,G,0.027270,0.021230,A,3.899,0.048320,1.2920,0.13020,1.00100,1.6680
257,5,chr5:110740674:G:T,110740674,T,0.027270,0.021230,G,3.899,0.048320,1.2920,0.13020,1.00100,1.6680
543,5,chr5:110744716:G:T,110744716,T,0.001136,0.000000,G,5.570,0.018280,NaN,NaN,NaN,NaN
740,5,chr5:110747478:G:A,110747478,A,0.002840,0.001022,G,4.274,0.038710,2.7820,0.51680,1.01100,7.6610
881,5,chr5:110749863:T:C,110749863,C,0.184800,0.202200,T,4.884,0.027110,0.8943,0.05054,0.81000,0.9875
965,5,chr5:110751049:C:A,110751049,A,0.004591,0.007626,C,3.854,0.049630,0.6002,0.26290,0.35850,1.0050
1058,5,chr5:110752280:C:T,110752280,T,0.002857,0.006240,C,6.446,0.011120,0.4564,0.31690,0.24520,0.8492


In [66]:
#Get the IDs to extract
sig_all_nonadj_id = sig_all_nonadj[['SNP']]
sig_all_nonadj_id
sig_all_nonadj_id.to_csv(f'{WORK_DIR}/{ancestry}.sig_all_nonadj_id.txt', sep = '\t', index=False, header=None)

In [67]:
## check to make sure file was created and saved
! ls {WORK_DIR}

chr5_EAS_release7.log
chr5_EAS_release7.pgen
chr5_EAS_release7.psam
chr5_EAS_release7.pvar
EAS_covariate_file.txt
EAS_release7.eigenvec
EAS_release7.related
EAS.samplestoKeep
EAS.sig_all_nonadj_id.txt
EAS_SLC25A46.all.assoc
EAS_SLC25A46.all_coding_nonsyn.variantstoKeep.txt
EAS_SLC25A46.all_coding.variantstoKeep
EAS_SLC25A46.all.log
EAS_SLC25A46.annovar.avinput
EAS_SLC25A46.annovar.hg38_multianno.txt
EAS_SLC25A46.annovar.hg38_multianno.vcf
EAS_SLC25A46.bed
EAS_SLC25A46.bim
EAS_SLC25A46.fam
EAS_SLC25A46.log
EAS_SLC25A46.vcf.gz
EAS_SLC25A46.vcf.gz.tbi
master_key_release7_final.csv


In [68]:
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
# Also extract the significant variants 
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract {WORK_DIR}/{ancestry}.sig_all_nonadj_id.txt \
--recode A \
--out {WORK_DIR}/{ancestry}_SLC25A46.all.nonadj

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.all.nonadj.log.
Options in effect:
  --bfile SLC25A46_EAS/EAS_SLC25A46
  --extract SLC25A46_EAS/EAS.sig_all_nonadj_id.txt
  --keep SLC25A46_EAS/EAS.samplestoKeep
  --out SLC25A46_EAS/EAS_SLC25A46.all.nonadj
  --recode A

3676 MB RAM detected; reserving 1838 MB for main workspace.
1951 variants loaded from .bim file.
5139 people (3269 males, 1870 females) loaded from .fam.
5099 phenotype values loaded from .fam.
--extract: 26 variants remaining.
--keep: 5138 people remaining.
Using 1 thread.
Before main variant filters, 5138 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [ ]:
recode = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.all.nonadj.raw', delim_whitespace=True)
recode

In [70]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in {ancestry} for SLC25A46: {len(variants)}')
variants

Number of variants in EAS for SLC25A46: 26


['chr5:110738976:T:G_G',
 'chr5:110739078:G:C_C',
 'chr5:110739776:AG:A_A',
 'chr5:110740651:A:G_G',
 'chr5:110740674:G:T_T',
 'chr5:110744716:G:T_T',
 'chr5:110747478:G:A_A',
 'chr5:110749863:T:C_C',
 'chr5:110751049:C:A_A',
 'chr5:110752280:C:T_T',
 'chr5:110752547:T:G_G',
 'chr5:110754208:A:G_G',
 'chr5:110754619:C:A_A',
 'chr5:110756460:A:G_G',
 'chr5:110756711:C:T_T',
 'chr5:110756831:A:G_G',
 'chr5:110757276:A:C_C',
 'chr5:110758239:A:G_G',
 'chr5:110759729:C:T_T',
 'chr5:110760413:T:A_A',
 'chr5:110761239:G:A_A',
 'chr5:110761857:G:A_A',
 'chr5:110762700:A:G_G',
 'chr5:110762932:T:C_C',
 'chr5:110764363:C:T_T',
 'chr5:110764756:G:C_C']

In [71]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
    freq_cases = (hom_cases + het_cases) / total_cases

    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
    freq_controls = (hom_controls + het_controls) / total_controls

    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Total Cases': total_cases,
        'Carrier freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Total Controls': total_controls,
        'Carrier freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['SNP'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

df_results

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,SNP
0,chr5:110738976:T:G_G,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110738976:T:G
1,chr5:110739078:G:C_C,0,0,2645,0.000000,0,7,2453,0.002854,chr5:110739078:G:C
2,chr5:110739776:AG:A_A,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110739776:AG:A
3,chr5:110740651:A:G_G,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110740651:A:G
4,chr5:110740674:G:T_T,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110740674:G:T
5,chr5:110744716:G:T_T,0,6,2645,0.002268,0,0,2453,0.000000,chr5:110744716:G:T
6,chr5:110747478:G:A_A,0,15,2645,0.005671,0,5,2453,0.002038,chr5:110747478:G:A
7,chr5:110749863:T:C_C,88,788,2645,0.331191,97,784,2453,0.359152,chr5:110749863:T:C
8,chr5:110751049:C:A_A,1,22,2645,0.008696,0,37,2453,0.015084,chr5:110751049:C:A
9,chr5:110752280:C:T_T,0,15,2645,0.005671,0,30,2453,0.012230,chr5:110752280:C:T


In [72]:
#Merge with the assoc file
sig_merge = sig_all_nonadj[['SNP','A1','F_A','F_U','A2','L95','OR','U95','P']]
merged = pd.merge(df_results, sig_merge, on='SNP', how='right')
merged

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,SNP,A1,F_A,F_U,A2,L95,OR,U95,P
0,chr5:110738976:T:G_G,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110738976:T:G,G,0.027300,0.021250,T,1.00200,1.2930,1.6690,0.048030
1,chr5:110739078:G:C_C,0,0,2645,0.000000,0,7,2453,0.002854,chr5:110739078:G:C,C,0.000000,0.001428,G,0.00000,0.0000,NaN,0.005989
2,chr5:110739776:AG:A_A,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110739776:AG:A,A,0.027270,0.021240,AG,1.00100,1.2920,1.6680,0.048690
3,chr5:110740651:A:G_G,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110740651:A:G,G,0.027270,0.021230,A,1.00100,1.2920,1.6680,0.048320
4,chr5:110740674:G:T_T,1,142,2645,0.054064,1,102,2453,0.041989,chr5:110740674:G:T,T,0.027270,0.021230,G,1.00100,1.2920,1.6680,0.048320
5,chr5:110744716:G:T_T,0,6,2645,0.002268,0,0,2453,0.000000,chr5:110744716:G:T,T,0.001136,0.000000,G,NaN,NaN,NaN,0.018280
6,chr5:110747478:G:A_A,0,15,2645,0.005671,0,5,2453,0.002038,chr5:110747478:G:A,A,0.002840,0.001022,G,1.01100,2.7820,7.6610,0.038710
7,chr5:110749863:T:C_C,88,788,2645,0.331191,97,784,2453,0.359152,chr5:110749863:T:C,C,0.184800,0.202200,T,0.81000,0.8943,0.9875,0.027110
8,chr5:110751049:C:A_A,1,22,2645,0.008696,0,37,2453,0.015084,chr5:110751049:C:A,A,0.004591,0.007626,C,0.35850,0.6002,1.0050,0.049630
9,chr5:110752280:C:T_T,0,15,2645,0.005671,0,30,2453,0.012230,chr5:110752280:C:T,T,0.002857,0.006240,C,0.24520,0.4564,0.8492,0.011120


In [73]:
## to CSV
merged.to_csv(f'{WORK_DIR}/{ancestry}_all-sign.variants-nonadj.txt', sep = '\t', index=False)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_all-sign.variants-nonadj.txt {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/{ancestry}_all-sign.variants-nonadj.txt')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/')

#### Logistic regression (glm)

In [ ]:
#Run with covariates
WORK_DIR = f'SLC25A46_{ancestry}'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--glm \
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/{ancestry}_SLC25A46.all_adj

In [77]:
assoc = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.all_adj.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
significant = assoc_add[assoc_add['P']<0.05]
significant

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
8368,5,110752057,chr5:110752057:C:T,C,T,Y,T,C,0.001402,N,ADD,2496,8.958330,0.883660,2.48125,0.013092,.
8464,5,110752280,chr5:110752280:C:T,C,T,Y,T,C,0.005689,N,ADD,2461,0.272875,0.592551,-2.19178,0.028396,.
13104,5,110760334,chr5:110760334:T:C,T,C,Y,C,T,0.000805,Y,ADD,2484,0.037578,1.496740,-2.19231,0.028357,.


In [78]:
#Get the IDs to extract
sig_all_adj_id = significant[['ID']]
sig_all_adj_id
sig_all_adj_id.to_csv(f'{WORK_DIR}/{ancestry}.sig_all_adj_id.txt', sep = '\t', index=False, header=None)

In [ ]:
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
# Also extract the significant variants 
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract {WORK_DIR}/{ancestry}.sig_all_adj_id.txt \
--recode A \
--out {WORK_DIR}/{ancestry}_SLC25A46.all.adj


In [ ]:
recode = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.all.adj.raw', delim_whitespace=True)
recode

In [81]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in {ancestry} for SLC25A46: {len(variants)}')
variants

Number of variants in EAS for SLC25A46: 3


['chr5:110752057:C:T_T', 'chr5:110752280:C:T_T', 'chr5:110760334:T:C_C']

In [82]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
    freq_cases = (hom_cases + het_cases) / total_cases

    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
    freq_controls = (hom_controls + het_controls) / total_controls

    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Total Cases': total_cases,
        'Carrier freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Total Controls': total_controls,
        'Carrier freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

df_results

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,ID
0,chr5:110752057:C:T_T,0,7,2645,0.002647,0,5,2453,0.002038,chr5:110752057:C:T
1,chr5:110752280:C:T_T,0,15,2645,0.005671,0,30,2453,0.012230,chr5:110752280:C:T
2,chr5:110760334:T:C_C,0,2,2645,0.000756,0,7,2453,0.002854,chr5:110760334:T:C


In [83]:
#Merge with the glm file
sig_merge = significant[['ID','A1','A1_FREQ','OBS_CT','OR','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
merged

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,ID,A1,A1_FREQ,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P
0,chr5:110752057:C:T_T,0,7,2645,0.002647,0,5,2453,0.002038,chr5:110752057:C:T,T,0.001402,2496,8.958330,0.883660,2.48125,0.013092
1,chr5:110752280:C:T_T,0,15,2645,0.005671,0,30,2453,0.012230,chr5:110752280:C:T,T,0.005689,2461,0.272875,0.592551,-2.19178,0.028396
2,chr5:110760334:T:C_C,0,2,2645,0.000756,0,7,2453,0.002854,chr5:110760334:T:C,C,0.000805,2484,0.037578,1.496740,-2.19231,0.028357


In [84]:
## to CSV
merged.to_csv(f'{WORK_DIR}/{ancestry}_all-sign.variants-adj.txt', sep = '\t', index=False)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_all-sign.variants-adj.txt {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/{ancestry}_all-sign.variants-adj.txt')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/')

### CODING

#### Association

In [87]:
WORK_DIR = f'SLC25A46_{ancestry}'

! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract range {WORK_DIR}/{ancestry}_SLC25A46.all_coding.variantstoKeep \
--assoc \
--allow-no-sex \
--ci 0.95 \
--out {WORK_DIR}/{ancestry}_SLC25A46.coding

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.coding.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile SLC25A46_EAS/EAS_SLC25A46
  --ci 0.95
  --extract range SLC25A46_EAS/EAS_SLC25A46.all_coding.variantstoKeep
  --keep SLC25A46_EAS/EAS.samplestoKeep
  --out SLC25A46_EAS/EAS_SLC25A46.coding

3676 MB RAM detected; reserving 1838 MB for main workspace.
1951 variants loaded from .bim file.
5139 people (3269 males, 1870 females) loaded from .fam.
5099 phenotype values loaded from .fam.
--extract range: 1942 variants excluded.
--extract range: 9 variants remaining.
--keep: 5138 people remaining.
Using 1 thread.
Before main variant filters, 5138 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%

In [88]:
freq = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.coding.assoc', delim_whitespace=True)
#freq[freq['P']<0.05]
freq

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,5,chr5:110739207:G:C,110739207,C,0.000189,0.000204,G,0.002846,0.95750,0.9273,1.4140,0.05799,14.8300
1,5,chr5:110739266:C:T,110739266,T,0.014240,0.011640,C,1.343000,0.24640,1.2270,0.1769,0.86760,1.7350
2,5,chr5:110739387:C:A,110739387,A,0.000189,0.000204,C,0.002763,0.95810,0.9284,1.4140,0.05805,14.8500
3,5,chr5:110743749:T:C,110743749,C,0.014850,0.013950,T,0.144200,0.70410,1.0660,0.1671,0.76790,1.4780
4,5,chr5:110743781:A:G,110743781,G,0.000378,0.000000,A,1.856000,0.17310,NaN,NaN,NaN,NaN
5,5,chr5:110748258:G:A,110748258,A,0.000000,0.000409,G,2.159000,0.14170,0.0000,inf,0.00000,NaN
6,5,chr5:110756711:C:T,110756711,T,0.000190,0.001637,C,5.980000,0.01447,0.1160,1.0610,0.01451,0.9281
7,5,chr5:110761239:G:A,110761239,A,0.183200,0.200300,G,4.757000,0.02919,0.8957,0.0505,0.81130,0.9889
8,5,chr5:110761398:G:C,110761398,C,0.000189,0.000408,G,0.414100,0.51990,0.4634,1.2250,0.04201,5.1120


In [89]:
#Get the IDs to extract
#sig_all_nonadj_id = sig_all_nonadj[['SNP']]
#sig_all_nonadj_id
#sig_all_nonadj_id.to_csv(f'{WORK_DIR}/{ancestry}.sig_all_nonadj_id.txt', sep = '\t', index=False, header=None)

In [90]:
## check to make sure file was created and saved
#! ls {WORK_DIR}

In [91]:
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
# Also extract the significant variants 
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract range SLC25A46_{ancestry}/{ancestry}_SLC25A46.all_coding.variantstoKeep \
--recode A \
--out {WORK_DIR}/{ancestry}_SLC25A46.coding.nonadj

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.coding.nonadj.log.
Options in effect:
  --bfile SLC25A46_EAS/EAS_SLC25A46
  --extract range SLC25A46_EAS/EAS_SLC25A46.all_coding.variantstoKeep
  --keep SLC25A46_EAS/EAS.samplestoKeep
  --out SLC25A46_EAS/EAS_SLC25A46.coding.nonadj
  --recode A

3676 MB RAM detected; reserving 1838 MB for main workspace.
1951 variants loaded from .bim file.
5139 people (3269 males, 1870 females) loaded from .fam.
5099 phenotype values loaded from .fam.
--extract range: 1942 variants excluded.
--extract range: 9 variants remaining.
--keep: 5138 people remaining.
Using 1 thread.
Before main variant filters, 5138 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%

In [ ]:
recode = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.coding.nonadj.raw', delim_whitespace=True)
recode

In [93]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in {ancestry} for SLC25A46: {len(variants)}')
variants

Number of variants in EAS for SLC25A46: 9


['chr5:110739207:G:C_C',
 'chr5:110739266:C:T_T',
 'chr5:110739387:C:A_A',
 'chr5:110743749:T:C_C',
 'chr5:110743781:A:G_G',
 'chr5:110748258:G:A_A',
 'chr5:110756711:C:T_T',
 'chr5:110761239:G:A_A',
 'chr5:110761398:G:C_C']

In [94]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
    freq_cases = (hom_cases + het_cases) / total_cases

    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
    freq_controls = (hom_controls + het_controls) / total_controls

    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Total Cases': total_cases,
        'Carrier freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Total Controls': total_controls,
        'Carrier freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['SNP'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

df_results

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,SNP
0,chr5:110739207:G:C_C,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739207:G:C
1,chr5:110739266:C:T_T,0,75,2645,0.028355,1,55,2453,0.022829,chr5:110739266:C:T
2,chr5:110739387:C:A_A,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739387:C:A
3,chr5:110743749:T:C_C,0,78,2645,0.029490,1,66,2453,0.027313,chr5:110743749:T:C
4,chr5:110743781:A:G_G,0,2,2645,0.000756,0,0,2453,0.000000,chr5:110743781:A:G
5,chr5:110748258:G:A_A,0,0,2645,0.000000,0,2,2453,0.000815,chr5:110748258:G:A
6,chr5:110756711:C:T_T,0,1,2645,0.000378,0,8,2453,0.003261,chr5:110756711:C:T
7,chr5:110761239:G:A_A,87,792,2645,0.332325,98,779,2453,0.357521,chr5:110761239:G:A
8,chr5:110761398:G:C_C,0,1,2645,0.000378,0,2,2453,0.000815,chr5:110761398:G:C


In [95]:
#Merge with the assoc file
sig_merge = freq[['SNP','A1','F_A','F_U','A2','L95','OR','U95','P']]
merged = pd.merge(df_results, sig_merge, on='SNP', how='right')
merged

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,SNP,A1,F_A,F_U,A2,L95,OR,U95,P
0,chr5:110739207:G:C_C,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739207:G:C,C,0.000189,0.000204,G,0.05799,0.9273,14.8300,0.95750
1,chr5:110739266:C:T_T,0,75,2645,0.028355,1,55,2453,0.022829,chr5:110739266:C:T,T,0.014240,0.011640,C,0.86760,1.2270,1.7350,0.24640
2,chr5:110739387:C:A_A,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739387:C:A,A,0.000189,0.000204,C,0.05805,0.9284,14.8500,0.95810
3,chr5:110743749:T:C_C,0,78,2645,0.029490,1,66,2453,0.027313,chr5:110743749:T:C,C,0.014850,0.013950,T,0.76790,1.0660,1.4780,0.70410
4,chr5:110743781:A:G_G,0,2,2645,0.000756,0,0,2453,0.000000,chr5:110743781:A:G,G,0.000378,0.000000,A,NaN,NaN,NaN,0.17310
5,chr5:110748258:G:A_A,0,0,2645,0.000000,0,2,2453,0.000815,chr5:110748258:G:A,A,0.000000,0.000409,G,0.00000,0.0000,NaN,0.14170
6,chr5:110756711:C:T_T,0,1,2645,0.000378,0,8,2453,0.003261,chr5:110756711:C:T,T,0.000190,0.001637,C,0.01451,0.1160,0.9281,0.01447
7,chr5:110761239:G:A_A,87,792,2645,0.332325,98,779,2453,0.357521,chr5:110761239:G:A,A,0.183200,0.200300,G,0.81130,0.8957,0.9889,0.02919
8,chr5:110761398:G:C_C,0,1,2645,0.000378,0,2,2453,0.000815,chr5:110761398:G:C,C,0.000189,0.000408,G,0.04201,0.4634,5.1120,0.51990


In [96]:
## to CSV
merged.to_csv(f'{WORK_DIR}/{ancestry}_coding.variants-nonadj.txt', sep = '\t', index=False)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_coding.variants-nonadj.txt {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/{ancestry}_coding.variants-nonadj.txt')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/')

#### Logistic regression (glm)

In [ ]:
#No significant SNPs with/without AGE as covariate
WORK_DIR = f'SLC25A46_{ancestry}'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/{ancestry}_SLC25A46 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract range {WORK_DIR}/{ancestry}_SLC25A46.all_coding.variantstoKeep \
--glm \
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/{ancestry}_SLC25A46.coding

In [100]:
assoc = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.coding.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
#assoc_add[assoc_add['P']<0.05]
assoc_add

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
0,5,110739207,chr5:110739207:G:C,G,C,Y,C,G,0.000400,N,ADD,2501,1.692520,1.473670,0.357082,0.721030,.
8,5,110739266,chr5:110739266:C:T,C,T,Y,T,C,0.014842,N,ADD,2493,0.968072,0.279820,-0.115963,0.907682,.
16,5,110739387,chr5:110739387:C:A,C,A,Y,A,C,0.000200,Y,ADD,2498,0.432218,1.636960,-0.512429,0.608351,.
24,5,110743749,chr5:110743749:T:C,T,C,Y,C,T,0.014889,N,ADD,2485,0.725545,0.275532,-1.164410,0.244258,.
32,5,110743781,chr5:110743781:A:G,A,G,Y,G,A,0.000400,Y,ADD,2499,13.420500,1.635000,1.588240,0.112232,.
40,5,110748258,chr5:110748258:G:A,G,A,Y,A,G,0.000401,Y,ADD,2494,0.580422,1.553570,-0.350162,0.726217,.
48,5,110756711,chr5:110756711:C:T,C,T,Y,T,C,0.000602,N,ADD,2490,0.334155,1.339390,-0.818393,0.413133,.
56,5,110761239,chr5:110761239:G:A,G,A,Y,A,G,0.193127,N,ADD,2488,1.105760,0.085167,1.180360,0.237856,.
64,5,110761398,chr5:110761398:G:C,G,C,Y,C,G,0.000200,Y,ADD,2502,4.512660,1.640980,0.918286,0.358469,.


In [101]:
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{ancestry}_SLC25A46\
--extract range {WORK_DIR}/{ancestry}_SLC25A46.all_coding.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--recode A \
--out {WORK_DIR}/{ancestry}_SLC25A46.coding

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to SLC25A46_EAS/EAS_SLC25A46.coding.log.
Options in effect:
  --bfile SLC25A46_EAS/EAS_SLC25A46
  --extract range SLC25A46_EAS/EAS_SLC25A46.all_coding.variantstoKeep
  --keep SLC25A46_EAS/EAS.samplestoKeep
  --out SLC25A46_EAS/EAS_SLC25A46.coding
  --recode A

3676 MB RAM detected; reserving 1838 MB for main workspace.
1951 variants loaded from .bim file.
5139 people (3269 males, 1870 females) loaded from .fam.
5099 phenotype values loaded from .fam.
--extract range: 1942 variants excluded.
--extract range: 9 variants remaining.
--keep: 5138 people remaining.
Using 1 thread.
Before main variant filters, 5138 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%

In [ ]:
recode = pd.read_csv(f'{WORK_DIR}/{ancestry}_SLC25A46.coding.raw', delim_whitespace=True)
recode

In [103]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in {ancestry} for SLC25A46: {len(variants)}')
variants

Number of variants in EAS for SLC25A46: 9


['chr5:110739207:G:C_C',
 'chr5:110739266:C:T_T',
 'chr5:110739387:C:A_A',
 'chr5:110743749:T:C_C',
 'chr5:110743781:A:G_G',
 'chr5:110748258:G:A_A',
 'chr5:110756711:C:T_T',
 'chr5:110761239:G:A_A',
 'chr5:110761398:G:C_C']

In [104]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
    freq_cases = (hom_cases + het_cases) / total_cases

    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
    freq_controls = (hom_controls + het_controls) / total_controls

    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Total Cases': total_cases,
        'Carrier freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Total Controls': total_controls,
        'Carrier freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

df_results

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,ID
0,chr5:110739207:G:C_C,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739207:G:C
1,chr5:110739266:C:T_T,0,75,2645,0.028355,1,55,2453,0.022829,chr5:110739266:C:T
2,chr5:110739387:C:A_A,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739387:C:A
3,chr5:110743749:T:C_C,0,78,2645,0.029490,1,66,2453,0.027313,chr5:110743749:T:C
4,chr5:110743781:A:G_G,0,2,2645,0.000756,0,0,2453,0.000000,chr5:110743781:A:G
5,chr5:110748258:G:A_A,0,0,2645,0.000000,0,2,2453,0.000815,chr5:110748258:G:A
6,chr5:110756711:C:T_T,0,1,2645,0.000378,0,8,2453,0.003261,chr5:110756711:C:T
7,chr5:110761239:G:A_A,87,792,2645,0.332325,98,779,2453,0.357521,chr5:110761239:G:A
8,chr5:110761398:G:C_C,0,1,2645,0.000378,0,2,2453,0.000815,chr5:110761398:G:C


In [105]:
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','OR','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
merged

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,ID,A1,A1_FREQ,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P
0,chr5:110739207:G:C_C,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739207:G:C,C,0.000400,2501,1.692520,1.473670,0.357082,0.721030
1,chr5:110739266:C:T_T,0,75,2645,0.028355,1,55,2453,0.022829,chr5:110739266:C:T,T,0.014842,2493,0.968072,0.279820,-0.115963,0.907682
2,chr5:110739387:C:A_A,0,1,2645,0.000378,0,1,2453,0.000408,chr5:110739387:C:A,A,0.000200,2498,0.432218,1.636960,-0.512429,0.608351
3,chr5:110743749:T:C_C,0,78,2645,0.029490,1,66,2453,0.027313,chr5:110743749:T:C,C,0.014889,2485,0.725545,0.275532,-1.164410,0.244258
4,chr5:110743781:A:G_G,0,2,2645,0.000756,0,0,2453,0.000000,chr5:110743781:A:G,G,0.000400,2499,13.420500,1.635000,1.588240,0.112232
5,chr5:110748258:G:A_A,0,0,2645,0.000000,0,2,2453,0.000815,chr5:110748258:G:A,A,0.000401,2494,0.580422,1.553570,-0.350162,0.726217
6,chr5:110756711:C:T_T,0,1,2645,0.000378,0,8,2453,0.003261,chr5:110756711:C:T,T,0.000602,2490,0.334155,1.339390,-0.818393,0.413133
7,chr5:110761239:G:A_A,87,792,2645,0.332325,98,779,2453,0.357521,chr5:110761239:G:A,A,0.193127,2488,1.105760,0.085167,1.180360,0.237856
8,chr5:110761398:G:C_C,0,1,2645,0.000378,0,2,2453,0.000815,chr5:110761398:G:C,C,0.000200,2502,4.512660,1.640980,0.918286,0.358469


In [106]:
## to CSV
merged.to_csv(f'{WORK_DIR}/{ancestry}_coding-variants-adj.txt', sep = '\t', index=False)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_coding-variants-adj.txt {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/{ancestry}_coding-variants-adj.txt')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/SLC25A46_{ancestry}/')